# Исследование алгоритма BoVW для нашей задачи

In [2]:
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
import yaml
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC

with open('config.yaml', 'r') as config:
    cfg = yaml.safe_load(config)

In [3]:
from BoVW.BoVW import BoVW
from dataset.get_pictures import DatasetOperations
from CustomDescriptors.abstract.abstract import ABSDescriptor
from CustomDescriptors.AkazeDescriptor.AKAZE import AKAZE
from CustomDescriptors.OrbDescriptor.ORB import ORB
from CustomDescriptors.ResnetDescriptor.Resnet import Resnet
from CustomDescriptors.SiftDescriptor.SIFT import SIFT
from helper.helper import Research, Multiprocessor

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import DBSCAN, KMeans, MiniBatchKMeans, AffinityPropagation, MeanShift, SpectralBiclustering, OPTICS, Birch


In [1]:
DatasetOperations.clear()

NameError: name 'DatasetOperations' is not defined

In [15]:
Research.add_train_dataset(percentage=10)

скопировано 66 картинок в train


In [18]:
Research.safe(
    "test",
    Research.test(
        Research.train(clf=DecisionTreeClassifier(min_samples_leaf=30, max_depth=5))
    )
)

found 82 keypoints
found 175 keypoints
found 80 keypoints
found 173 keypoints
found 199 keypoints
